In [1]:
import ee

ee.Initialize()

In [2]:
from cnwi import sar, datacube, fourier, datasets, rf, trainingd as td

In [3]:
ns_data_set = datasets.NovaScotia()
    
ee_training = ee.FeatureCollection("projects/fpca-336015/assets/NovaScotia/_527_POINTS")
eegrid = ee.FeatureCollection("projects/fpca-336015/assets/NovaScotia/_527_ECO_DIST").geometry()

# Sentinel -1 
s1_col = ee.ImageCollection(ns_data_set.sentienl1)
sar_inpts = sar.build_s1_inputs(s1_col)
alos_inpts = sar.build_alos_inpts(2019)

# Data Cube
dc_col = ee.ImageCollection(ns_data_set.data_cube)
dcpp = datacube.prep_data_cube(dc_col)
dc_inpts = datacube.build_data_cube_inpts(dcpp)


# Terrain Analysis
elev_inpt = ee.ImageCollection(ns_data_set.terrain_analysis).mosaic()

# create fourier transform
fourier = ee.ImageCollection("projects/fpca-336015/assets/NovaScotia/fourier_transform").mosaic()

# create the stack
stack = ee.Image.cat(sar_inpts, alos_inpts, dc_inpts, elev_inpt, fourier)

training_prep = td.prep_training_data(col=ee_training, class_property='land_cover')
training_samples = td.generate_samples(col=training_prep, stack=stack)

clf = rf.RandomForestModel(
    numberOfTrees=1000
)

trained = clf.train(
    training_data=training_samples,
    predictors=stack.bandNames(),
    classProperty='value'
)

clf_img = stack.classify(trained).uint8().clip(eegrid)

AttributeError: 'ImageCollection' object has no attribute 'bandNames'

In [ ]:
import geemap

In [ ]:
Map = geemap.Map()

Map.addLayer(clf_img, {'min': 1, 'max': 6})
Map.centerObject(clf_img, 12)

Map

Map(center=[45.53836464965929, -62.28521924329163], controls=(WidgetControl(options=['position', 'transparent_…